In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
artist_profiles = pd.read_csv('artists.csv')
user_profiles = pd.read_csv('profiles.csv')

In [3]:
df = pd.concat([user_profiles, pd.get_dummies(user_profiles['sex'], dummy_na=True)], axis=1)
df.rename(columns = lambda x: str(x), inplace=True)
df.rename(columns = {'nan': 'sex-nan'}, inplace=True)
del df['sex']

df['age-nan'] = np.isnan(df['age'])*1
nans = np.isnan(df['age'])
df.loc[nans,'age'] = 0
df['age-negative'] = (df['age']<0)*1
negatives = df['age-negative']==1
df.loc[negatives,'age'] = 0
df['age-toohigh'] = (df['age']>120)*1
toohighs = df['age-toohigh']==1
df.loc[toohighs,'age'] = 0

df = pd.concat([df, pd.get_dummies(df['country'], dummy_na=False)], axis=1)
del df['country']

df = df.set_index('user')
df.index.name = None
df.head()

,age,f,m,sex-nan,age-nan,age-negative,age-toohigh,Afghanistan,Albania,Algeria,...,Vanuatu,Venezuela,Viet Nam,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5909125332c108365a26ccf0ee62636eee08215c,29,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,30,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63268cce0d68127729890c1691f62d5be5abd87c,21,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02871cd952d607ba69b64e2e107773012c708113,24,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
user_dict = {}
for i in train['user']:
    user_dict[i] = {}

In [17]:
%%time
for i in train['user']:
    x = train.loc[train['user'] == i]
    for j in x:
        user_dict[i] = dict(zip(x.artist, x.plays))

KeyboardInterrupt: 

In [ ]:
artistdf = pd.DataFrame.from_dict(user_dict, orient='index')
nans = np.isnan(artistdf)
artistdf[nans] = 0
artistdf

In [ ]:
fulldf = pd.concat([df,artistdf],axis=1)
fulldf.head(10)

In [22]:
%%time
user_means = {}
for index,mean in train.groupby('user')['plays'].mean().iteritems():
    user_means[index] = mean

Wall time: 3.05 s


In [10]:
import scipy.sparse as sps

In [12]:
mtx = sps.dok_matrix((len(user_profiles), len(artist_profiles)), dtype=np.float64)
mtx

<233286x2000 sparse matrix of type '<type 'numpy.float64'>'
	with 0 stored elements in Dictionary Of Keys format>

In [13]:
train.head()

,user,artist,plays
0,eb1c57ddc9e0e2d005169d3a1a96e8dd95e3af03,5a8e07d5-d932-4484-a7f7-e700793a9c94,554
1,44ce793a6cd9d20f13f4a576a818ef983314bb5d,a3a92047-be1c-4f3e-8960-c4f8570984df,81
2,da9cf3f557161d54b76f24db64be9cc76db008e3,eeb1195b-f213-4ce1-b28c-8565211f8e43,708
3,8fa49ab25d425edcf05d44bfc1d5aea895287d81,a1419808-65d3-4d40-998c-1a0bac65eabc,265
4,b85fcaef67d2669cd99b334b5e8c8705263db2cf,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,220


In [ ]:
for ir in range(len(user_profiles)):
    for ic in range(len(artist_profiles)):
        mtx[ir, ic] = 1.0 * (ir != ic)

In [14]:
s = pd.Series([3.0, np.nan, 1.0, 3.0, np.nan, np.nan])

In [15]:
s.index = pd.MultiIndex.from_tuples([(1, 2, 'a', 0),
                                         (1, 2, 'a', 1),
                                         (1, 1, 'b', 0),
                                         (1, 1, 'b', 1),
                                         (2, 1, 'b', 0),
                                         (2, 1, 'b', 1)],
                                         names=['A', 'B', 'C', 'D'])

In [16]:
s

A  B  C  D
1  2  a  0     3
         1   NaN
   1  b  0     1
         1     3
2  1  b  0   NaN
         1   NaN
dtype: float64

In [ ]:
arrays = [list(train['user']),list(train['artist'])]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['user', 'artist'])
index